In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
# from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [132]:
# From Kaggle Competition. Expedia Hotel Recommendations
# https://www.kaggle.com/c/expedia-hotel-recommendations

# load the data from expedia
# destinations = pd.read_csv('../Final_Project/data/destinations.csv') 
# Read data into pandas and explore
# expedia = pd.read_csv('../Final_Project/data/train.csv') # takes a LONG time to load the full dataset! 
expedia = pd.read_csv('../Final_Project/data/train.csv',nrows=10000) # test code with smaller set first for faster code check. 
# other dataset provided: testing dataset
# df_test = pd.read_csv('../Final_Project/data/test.csv'); # will read this is later when/if needed. 

In [133]:
print 'Dataframe Shape'
print expedia.shape
expedia.head(2)
# destinations.shape
# destinations.head(2)

Dataframe Shape
(10000, 24)


,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1


Explaination of Columns from https://www.kaggle.com/c/expedia-hotel-recommendations/data 

date_time - Timestamp - string
site_name - ID of the Expedia point of sale - int
posa_continent - ID of continent associated with site_name	- int
user_location_country - The ID of the country the customer is located - int
user_location_region - The ID of the region the customer is located - int
user_location_city - The ID of the city the customer is located - int
orig_destination_distance - Physical distance between a hotel and a customer at the time of search. A null means the distance could not be calculated- double
user_id - ID of user - int
is_mobile - 1 when a user connected from a mobile device, 0 otherwise - tinyint
is_package - 1 if the click/booking was generated as a part of a package, 0 otherwise - int
channel  - Checkin date - string
srch_co - Checkout date - string
srch_adults_cnt - The number of adults specified in the hotel room - int
srch_children_cnt - The number of (extra occupancy) children specified in the hotel room - int
srch_rm_cnt - The number of hotel rooms specified in the search - int
srch_destination_id - ID of the destination where the hotel search was performed - int
srch_destination_type_id - Type of destination - int
hotel_continent - Hotel continent - int
hotel_country - Hotel country - int
hotel_market - Hotel market - int
is_booking - 1 if a booking, 0 if a click - tinyint
cnt - Numer of similar events in the context of the same user session  -bigint
hotel_cluster - ID of a hotel cluster - int

In [134]:
# Print some info about dataset

# user IDs. 
print 'Dataset Stats'
print 'Size of Dataframe', expedia.shape

# unique User counts. 
user = expedia.groupby('user_id').user_id.count()
muser = user.mean()
print 'Unique Users: ', len(user)
print 'Mean entries per user: ', muser

# Number of bookings vs. other entries. 
lbook = len(expedia.loc[(expedia['is_booking']  == 1)])
num_book = expedia.groupby('is_booking').is_booking.count()
print 'Number of entries where users book', lbook, 'total', len(expedia)

# unique hotel cluster counts. 
hc = expedia.groupby('hotel_cluster').hotel_cluster.count()
print 'Unique Hotel Cluter IDs: ', len(hc)

Dataset Stats
Size of Dataframe (10000, 24)
Unique Users:  339
Mean entries per user:  29.4985250737
Number of entries where users book 777 total 10000
Unique Hotel Cluter IDs:  100


In [126]:
# clean up some columns to use
# date time, srch_ci, srch_co, orig_destination_distance,

# type(expedia)
# df2['timestamp'] = pd.Timestamp('20120101')

In [127]:
# expedia.isnull().sum()
# orig_destination_distance,  srch_ci, srch_co,  contains null values
# expedia.loc[pd.isnull(expedia['orig_destination_distance'])] = np.nan
# expedia.orig_destination_distance
expedia.isnull().sum()

date_time                       0
site_name                       0
posa_continent                  0
user_location_country           0
user_location_region            0
user_location_city              0
orig_destination_distance    3729
user_id                         0
is_mobile                       0
is_package                      0
channel                         0
srch_ci                         7
srch_co                         7
srch_adults_cnt                 0
srch_children_cnt               0
srch_rm_cnt                     0
srch_destination_id             0
srch_destination_type_id        0
is_booking                      0
cnt                             0
hotel_continent                 0
hotel_country                   0
hotel_market                    0
hotel_cluster                   0
dtype: int64

In [128]:
# Predict if a person is booking or not. 
# We do not care what hotel cluster a person is looking at unless they book. 

# print expedia.hotel_cluster.max()
# print expedia.hotel_cluster.min()
# expedia.corr()


# labels = expedia.columns # features to train with 
# full list
# X = [ 
#     { 'date_time', 'site_name', 'posa_continent', 'user_location_country',
#        'user_location_region', 'user_location_city',
#        'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
#        'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt',
#        'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id',
#        'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_continent',
#        'hotel_country', 'hotel_market', 'hotel_cluster'
#      }]

X = expedia.drop(['is_booking','date_time','srch_ci','srch_co', 'orig_destination_distance'], axis = 1) # features
feature_cols = X.columns # get colnames for use later
# np.array(expedia) 
target = expedia.is_booking # y 

In [129]:
# add a null model for comparison of baseline 

mo_target = target.mode() # guess 0 - they do not book all the time. 
# pred = np.zero(len(expdedia),1)

In [130]:
# train a classification model with existing columns
# trying KNN and log reg to start. 
# Note: do not need to test_train_split because Kaggle provides a separate traing set. 

# knn
# k = 3
# knn = KNeighborsClassifier(n_neighbors=k)
# knn.fit(X, target)
# knn.score(X, target)

# from sklearn.grid_search import GridSearchCV
# k_range = range(2, 30, 1)
# param_grid = dict(n_neighbors=k_range)
# grid = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
# grid.fit(X_train, y_train)
# grid.grid_scores_
# grid_mean_scores = [result[1] for result in grid.grid_scores_] # get scores from grid.grid_scores_
# visualize results
# plt.figure()
# plt.plot(k_range, grid_mean_scores)

# retraining with added column
# knn
k = 3
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X, target)
knn.score(X, target)
preds_knn = knn.predict(X)
print 'Accuracy for KNN model'
print accuracy_score(target, preds_knn)
print 'KNN Confusion Matrix'
print metrics.confusion_matrix(target, preds_knn)
target_names = ['Did not Book 0','Booked 1']
print(classification_report(target, preds_knn, target_names=target_names))

# test set 
# preds_knn = knn.predict(X_test)
# print 'Accuracy for KNN model'
# print accuracy_score(y_test, preds_knn)
# print 'KNN Confusion Matrix'
# print metrics.confusion_matrix(y_test, preds_knn)
# print(classification_report(y_test, preds_knn, target_names=target_names))


Accuracy for KNN model
0.9254
KNN Confusion Matrix
[[9153   70]
 [ 676  101]]
                precision    recall  f1-score   support

Did not Book 0       0.93      0.99      0.96      9223
      Booked 1       0.59      0.13      0.21       777

   avg / total       0.90      0.93      0.90     10000



In [131]:
# LogReg
logreg = LogisticRegression(C=1e9)
logreg.fit(X, target)
assorted_pred_class = logreg.predict(X)
logregm = zip(feature_cols,logreg.coef_[0])
preds_logreg = logreg.predict(X)
print 'Accuracy for Logistic Regression model'
print accuracy_score(target, preds_logreg)
print 'Logistic Regression Confusion Matrix'
print metrics.confusion_matrix(target, preds_logreg)
print(classification_report(target, preds_logreg, target_names=target_names))

Accuracy for Logistic Regression model
0.9223
Logistic Regression Confusion Matrix
[[9223    0]
 [ 777    0]]
                precision    recall  f1-score   support

Did not Book 0       0.92      1.00      0.96      9223
      Booked 1       0.00      0.00      0.00       777

   avg / total       0.85      0.92      0.89     10000



Switching to predicting which cluster 
Assuming we can use first part to 

In [112]:
# for predicting which hotel cluster the person will book. 

# Select df only when the user is booking the hotel. 
# only look at when a person is booking. 
df_book = expedia.loc[expedia.is_booking == 1]
print df_book.shape
# df_book.corr()

X = df_book.drop(['hotel_cluster','date_time','srch_ci','srch_co','orig_destination_distance'], axis = 1) # features
target = df_book.hotel_cluster

(99, 24)


In [113]:
# Null model

In [116]:
k = 3
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X, target)
knn.score(X, target)
preds_knn = knn.predict(X)
print 'Accuracy for KNN model'
print accuracy_score(target, preds_knn)
print 'KNN Confusion Matrix'
print metrics.confusion_matrix(target, preds_knn)
# target_names = ['Did not Book 0','Booked 1']
# print(classification_report(target, preds_knn, target_names=target_names))
print(classification_report(target, preds_knn))

Accuracy for KNN model
0.434343434343
KNN Confusion Matrix
[[2 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [0 0 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
             precision    recall  f1-score   support

          1       0.22      1.00      0.36         2
          2       0.00      0.00      0.00         1
          6       0.25      1.00      0.40         1
          7       0.33      1.00      0.50         2
          8       0.25      0.50      0.33         2
         10       0.38      1.00      0.55         3
         11       0.40      1.00      0.57         2
         12       1.00      1.00      1.00         2
         14       0.50      1.00      0.67         2
         15       1.00      1.00      1.00         2
         16       0.00      0.00      0.00         1
         17       0.33      1.00      0.50         1
         18       0.38      0.75      0.50         4
         19       0.50      0.50      0.50         2
         20      

In [119]:
logreg = LogisticRegression(C=1e9)
logreg.fit(X, target)
assorted_pred_class = logreg.predict(X)
logregm = zip(feature_cols,logreg.coef_[0])
preds_logreg = logreg.predict(X)
print 'Accuracy for Logistic Regression model'
print accuracy_score(target, preds_logreg)
print 'Logistic Regression Confusion Matrix'
print metrics.confusion_matrix(target, preds_logreg)
print(classification_report(target, preds_logreg))

Accuracy for Logistic Regression model
0.929292929293
Logistic Regression Confusion Matrix
[[2 0 0 ..., 0 0 0]
 [0 1 0 ..., 0 0 0]
 [0 0 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 0 ..., 0 0 2]]
             precision    recall  f1-score   support

          1       1.00      1.00      1.00         2
          2       1.00      1.00      1.00         1
          6       0.50      1.00      0.67         1
          7       1.00      1.00      1.00         2
          8       1.00      0.50      0.67         2
         10       1.00      1.00      1.00         3
         11       1.00      1.00      1.00         2
         12       1.00      1.00      1.00         2
         14       1.00      1.00      1.00         2
         15       0.67      1.00      0.80         2
         16       1.00      1.00      1.00         1
         17       1.00      1.00      1.00         1
         18       0.75      0.75      0.75         4
         19       1.00      1.00      

In [ ]:
# feature ideas: 
# searched hotel cluster AND is booking. 
# is family - adults = 2 and there are additional guests
# number or clicks before purchase
# repeat clicks? user visits same hotel cluster again 
# Weekend/Weekday (business or personal travel maybe) ( check in day of week. Check out day of week)
# if user books more than one hotel - is it in the same cluster, same location? 
# time on site. (I think this will be correlated to booking something)

In [ ]:
# test Full model
# combination of best first classification + predicting hotel cluster. 

